In [1]:
%%capture
!pip install dfcx-scrapi

## imports

In [61]:
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

# For connect to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g


import pandas as pd
import numpy as np

## g-sheets config

In [51]:
# service account scopes
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/christophertrauco/.config/secrets/cx_admin_sa.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '18qvh0uWkA0z7S5CC3xTdvD2wkhsmnINjgxUWmuoCo7M'
sh = client.open_by_key(spreadsheet_key)
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad


In [52]:
# object attribute mapping index
intents_ss = sh.worksheet('Index').acell('B1').value
print(intents_ss)

1lt02Hh2B_q_sluK-sAcgqyJ8WbqfZ6qDP_EQJVHPJDo


## agent object attribute mappings


In [53]:
# object data attribute mappings 
# master workbook @ https://itrau.co/bc-dfcx-sad-main

project_id = sh.worksheet('Main').acell('B2').value
agent_id = sh.worksheet('Main').acell('D3').value
display_name = sh.worksheet('Main').acell('B3').value
gcp_region = sh.worksheet('Main').acell('B4').value
zone = sh.worksheet('Main').acell('B5').value
language = sh.worksheet('Main').acell('B6').value
default_start_flow = sh.worksheet('Main').acell('B8').value
time_zone = sh.worksheet('Main').acell('B7').value
creds_path = sh.worksheet('Main').acell('B10').value
google_sheet_name = sh.worksheet('Main').acell('B11').value
google_sheet_tab_read = sh.worksheet('Main').acell('B12').value

intents_ss = sh.worksheet('Index').acell('B1').value

parent_value = sh.worksheet('Main').acell('B13').value
# print(project_id, display_name, gcp_region, gcp_region, zone, language, default_start_flow,time_zone)
print(parent_value)

 projects/bot1-404120/locations/global/agents/4a2fde83-decc-4656-8995-1cbc2e3eeab4


# output selected agent intents

In [46]:
from google.cloud import dialogflowcx_v3


def list_intents():
    # Create a client
    client = dialogflowcx_v3.IntentsClient()

    # Initialize request argument(s)
    request = dialogflowcx_v3.ListIntentsRequest(
        parent=agent_id,
    )

    # Make the request
    page_result = client.list_intents(request=request)

    # Handle the response
    for response in page_result:
        print(response)



In [56]:
%%capture
%%sh
./etl.sh

In [62]:
df = pd.read_csv('list_intents.csv')
# print(df)

wks_name = 'Intents'
cell_of_start_df = 'A1'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           intents_ss,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 685, saw 3


In [24]:
dffx = DataframeFunctions(creds_path)
# print(dffx)


In [25]:
df = dffx.sheets_to_dataframe(google_sheet_name, google_sheet_tab_read)

In [ ]:
dffx.bulk_update_intents_from_dataframe(agent_id, df, update_flag=True, rate_limiter=1)